<a href="https://colab.research.google.com/github/martush/martush_notebooks/blob/develop/Reddit_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import praw
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import numpy as np

In [ ]:
!pip install praw

  Using cached praw-7.5.0-py3-none-any.whl (176 kB)
  Using cached prawcore-2.3.0-py3-none-any.whl (16 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)


In [ ]:
# Setup needed for function which disables SSL verify
import warnings
import contextlib
import requests
from urllib3.exceptions import InsecureRequestWarning
old_merge_environment_settings = requests.Session.merge_environment_settings

In [ ]:
    @contextlib.contextmanager
    def no_ssl_verification():
        '''
        Custom utility function. Used to suppress SSL verify for
        all other functions which connect to Google Ads API.
        '''
        opened_adapters = set()

        def merge_environment_settings(self, url, proxies, stream, verify, cert):
            # Verification happens only once per connection so we need to close
            # all the opened adapters once we're done. Otherwise, the effects of
            # verify=False persist beyond the end of this context manager.
            opened_adapters.add(self.get_adapter(url))

            settings = old_merge_environment_settings(self, url, proxies, stream, verify, cert)
            settings['verify'] = False

            return settings

        requests.Session.merge_environment_settings = merge_environment_settings

        try:
            with warnings.catch_warnings():
                warnings.simplefilter('ignore', InsecureRequestWarning)
                yield
        finally:
            requests.Session.merge_environment_settings = old_merge_environment_settings

            for adapter in opened_adapters:
                try:
                    adapter.close()
                except:
                    pass

In [ ]:
reddit = praw.Reddit(
    client_id="HfEdm22XuoZrC29e5QHnXA",
    client_secret="h9y0iiUaCbImyuXg0Z21stIUohypPw",
    password="fxcm2022!",
    user_agent="testscript by u/fakebot3",
    username="RemarkablePapaya1093",
)

In [ ]:
with no_ssl_verification():
    print(reddit.user.me())

RemarkablePapaya1093


In [ ]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mivanova\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mivanova\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
sub_reddits = reddit.subreddit('wallstreetbets')
stocks = ["GME", "AMC"]

In [ ]:
def commentSentiment(ticker, urlT):
    subComments = []
    bodyComment = []
    try:
        check = reddit.submission(url=urlT)
        subComments = check.comments
    except:
        return 0

    for comment in subComments:
        try:
            bodyComment.append(comment.body)
        except:
            return 0

    sia = SIA()
    results = []
    for line in bodyComment:
        scores = sia.polarity_scores(line)
        scores['headline'] = line

        results.append(scores)

    df =pd.DataFrame.from_records(results)
    df.head()
    df['label'] = 0

    try:
        df.loc[df['compound'] > 0.1, 'label'] = 1
        df.loc[df['compound'] < -0.1, 'label'] = -1
    except:
        return 0

    averageScore = 0
    position = 0
    while position < len(df.label)-1:
        averageScore = averageScore + df.label[position]
        position += 1
    averageScore = averageScore/len(df.label)

    return(averageScore)

In [ ]:
def latestComment(ticker, urlT):
    subComments = []
    updateDates = []
    try:
        check = reddit.submission(url=urlT)
        subComments = check.comments
    except:
        return 0

    for comment in subComments:
        try:
            updateDates.append(comment.created_utc)
        except:
            return 0

    updateDates.sort()
    return(updateDates[-1])

In [ ]:
def get_date(date):
    return dt.datetime.fromtimestamp(date)

In [ ]:
reddit

In [ ]:
submission_statistics = []
d = {}
for ticker in stocks:
    with no_ssl_verification():
        for submission in reddit.subreddit('wallstreetbets').search(ticker, limit=130):
            if submission.domain != "self.wallstreetbets":
                continue
            d = {}
            d['ticker'] = ticker
            d['num_comments'] = submission.num_comments
            d['comment_sentiment_average'] = commentSentiment(ticker, submission.url)
            if d['comment_sentiment_average'] == 0.000000:
                continue
            d['latest_comment_date'] = latestComment(ticker, submission.url)
            d['score'] = submission.score
            d['upvote_ratio'] = submission.upvote_ratio
            d['date'] = submission.created_utc
            d['domain'] = submission.domain
            d['num_crossposts'] = submission.num_crossposts
            d['author'] = submission.author
            submission_statistics.append(d)

dfSentimentStocks = pd.DataFrame(submission_statistics)

_timestampcreated = dfSentimentStocks["date"].apply(get_date)
dfSentimentStocks = dfSentimentStocks.assign(timestamp = _timestampcreated)

_timestampcomment = dfSentimentStocks["latest_comment_date"].apply(get_date)
dfSentimentStocks = dfSentimentStocks.assign(commentdate = _timestampcomment)

dfSentimentStocks.sort_values("latest_comment_date", axis = 0, ascending = True,inplace = True, na_position ='last')

dfSentimentStocks

,ticker,num_comments,comment_sentiment_average,latest_comment_date,score,upvote_ratio,date,domain,num_crossposts,author,timestamp,commentdate
32,AMC,199,0.153846,1.636734e+09,822,0.88,1.635787e+09,self.wallstreetbets,0,Thump4,2021-11-01 19:09:32,2021-11-12 18:15:18
29,AMC,98,0.027778,1.641905e+09,114,0.79,1.641833e+09,self.wallstreetbets,0,j_stars,2022-01-10 18:37:01,2022-01-11 14:38:09
34,AMC,200,0.104167,1.642554e+09,218,0.68,1.636815e+09,self.wallstreetbets,0,Winter-Divide-8049,2021-11-13 16:45:51,2022-01-19 03:01:43
35,AMC,80,-0.130435,1.642602e+09,7,0.55,1.642554e+09,self.wallstreetbets,0,Boforizzle,2022-01-19 02:58:22,2022-01-19 16:22:40
28,AMC,80,0.026316,1.642622e+09,81,0.74,1.642539e+09,self.wallstreetbets,0,JETLIFEMUZIK94,2022-01-18 22:54:56,2022-01-19 21:48:06
25,AMC,172,0.042857,1.643780e+09,68,0.54,1.643720e+09,self.wallstreetbets,0,999smoothape,2022-02-01 15:00:08,2022-02-02 07:34:27
27,AMC,92,0.186047,1.644483e+09,27,0.55,1.644443e+09,self.wallstreetbets,0,BFA_Artist,2022-02-09 23:44:47,2022-02-10 10:56:53
24,AMC,77,-0.117647,1.644586e+09,122,0.63,1.644469e+09,self.wallstreetbets,0,orthonut20,2022-02-10 06:57:36,2022-02-11 15:24:32
30,AMC,33,-0.176471,1.644868e+09,14,0.55,1.644549e+09,self.wallstreetbets,0,999smoothape,2022-02-11 05:15:13,2022-02-14 21:52:28
20,AMC,144,-0.027778,1.645083e+09,112,0.65,1.645028e+09,self.wallstreetbets,0,BFA_Artist,2022-02-16 18:14:19,2022-02-17 09:29:50


In [ ]:
dfSentimentStocks.author.value_counts()

999smoothape           2
BFA_Artist             2
wetdirtkurt            2
Thump4                 1
xXdont_existxX         1
BisexualFag69          1
Secret_Hero_303        1
OB1KENOB               1
Freadom6               1
Independent-Ad4660     1
Krowfficial            1
Haters_Gunner_Hate     1
saliym1988             1
ketaking1976           1
TheRealJugger          1
kaankillerboy2         1
Njkoskin               1
BearsGonnaCOPE         1
Thorbeans              1
j_stars                1
jjd1226                1
Teekay53               1
silversnoopy           1
The_Average_Asshole    1
lilape90               1
JunkMan51              1
Banana_banana666       1
ares_abi               1
Everettrtucker         1
orthonut20             1
JETLIFEMUZIK94         1
Boforizzle             1
Winter-Divide-8049     1
girthygirthmonster     1
Name: author, dtype: int64

In [ ]:
dfSentimentStocks.to_csv('C:\Users\mivanova\Desktop\Reddit_Sentiment_Equity.csv', index=False)